<a href="https://colab.research.google.com/github/pnjha/NLP-Assignments/blob/master/NPL1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


In [2]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import copy
import math
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed, Bidirectional, Embedding
from keras.layers import RepeatVector
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout, Activation
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

Using TensorFlow backend.


In [3]:
X = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/dev.en', names=['src'])
Y = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/dev.hi', names=['dest'])
lines = pd.concat([X,Y], axis=1)
lines = shuffle(lines)
lines.sample(10)

,src,dest
160,Hundreds of such small creatures live in our b...,हमारे शरीर में सैकड़ों ऐसे छोटे जीव रहते हैं ज...
159,"Tuberculosis , pneumonia etc . encircle .",टी.बी. निमोनिया आदि घेर लेते हैं ।
140,Patients are administered more anesthetic medi...,"मरीज को ज्यादा निश्चेतक दवा देनी पड़ती है , जिस..."
38,Start with light exercise .,प्रारंभ सूक्ष्म व्यायाम से करो ।
138,These possibilities are negligible in laparosc...,लैपरोस्कोपिक विधि में ये संभावनाएँ नगण्य होती ...
81,Try to remember the new information by relatin...,नई जानकारी को किसी पुरानी जानकारी से जोड़कर याद...
65,Brain exercise works on this very fundamental .,ब्रेन एक्सरसाइज इसी फंडे पर काम करती है ।
299,During the cleaning of bus everyone is asked t...,बस की सफाई के दौरान सभी को बाहर जाने के लिए कह...
26,Including these things in your diet you can sa...,इन्ही चीजों को अपने आहार में शामिल कर आप आँखों...
289,"Actually , the early you reach , the better se...","दरअसल , जितना जल्दी आप पहुँचेंगे , उतनी अच्छी ..."


In [0]:
def process_data(data,append_char):
  data = data.apply(lambda x: x.lower())
  data = data.apply(lambda x: x.strip())
  data = data.apply(lambda x: re.sub("'", '', x))
  exclude = set(string.punctuation) # Set of all special characters
  data = data.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
  remove_digits = str.maketrans('', '', digits)
  data = data.apply(lambda x: x.translate(remove_digits))
  if append_char:
    data = data.apply(lambda x : '<SOS> '+ x + ' <EOS>')
  return data

In [5]:
lines.eng = process_data(lines.src,False)
lines.hin = process_data(lines.dest,True)
lines.sample(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


,src,dest
28,Taking diet rich in vitamin-A can also be help...,विटामिन-ए से भरपूर आहार लेना भी आँखों को स्वस्...
54,"Eat simple , easily digestible , nutritious fo...","सादा , सुपाच्य , पौष्टिक आहार खाएँ , चोकरयुक्त..."
297,Establish a familiarity with the bus driver an...,बस ड्राइवर व कंडक्टर से पहचान बना लें ।
257,Early morning I had seen a tourist from Hollan...,सुबह-सुबह मैंने हॉलैंड के एक पर्यटक को गहरे ध्...
113,The following consequences show up if a person...,किसी व्यक्ति को एड्स होने पर निम्न दुष्परिणाम ...
70,Read attentively whatever is to be remembered ...,जो भी याद रखना है उसे गौर से पढ़ें और आगे बढ़न...
315,Here unknown soldiers martyered in World War I...,यहाँ प्रथम विश्‍व युद्ध में शहीद हुए अज्ञात सै...
188,The main reason of blindness is cataract which...,दृष्टिहीनता का मुख्य कारण मोतियाबिंद है जो कुल...
360,No doubt they will show you a scene of a very ...,बेशक ये आपको एक अलग ही दुनिया का नजारा दिखाएँग...
18,The use of dry fruits etcetera in Indian cuisi...,भारतीय व्यंजनों में मेवे आदि का इस्तेमाल सदियो...


In [0]:
def get_vocab_list(data):
  vocab = []
  for line in data:
    temp = line.split()
    for item in temp:
      if item not in vocab:
        vocab.append(item)
  return vocab

In [0]:
src_vocab = get_vocab_list(lines.src)
dest_vocab = get_vocab_list(lines.dest)

In [8]:
print(src_vocab[:10])
print(dest_vocab[:10]) 

['It', 'is', 'worth', 'noting', 'that', 'this', 'road', 'the', 'main', 'way']
['गौरतलब', 'है', 'कि', 'यह', 'सड़क', 'रोम', 'का', 'मुख्य', 'मार्ग', '।']


In [0]:
# Max Length of source sequence
def get_max_sentence_size(data):
  max_len = 0
  for line in data:
    temp = line.split()
    max_len = max(max_len, len(temp))
  return max_len

In [10]:
src_max_len = get_max_sentence_size(lines.src)
dest_max_len = get_max_sentence_size(lines.dest)
src_max_len,dest_max_len

(50, 48)

In [11]:
src_vocab_len = len(src_vocab)
dest_vocab_len = len(dest_vocab) + 1
src_vocab_len, dest_vocab_len

(1867, 2009)

In [0]:
def get_word_to_index_dict(vocab):
  word_to_index = {}
  for i, word in enumerate(vocab):
    word_to_index[word] = i
  return word_to_index

In [0]:
src_word_to_index = get_word_to_index_dict(src_vocab)
dest_word_to_index = get_word_to_index_dict(dest_vocab)

In [14]:
print(src_word_to_index)
print(dest_word_to_index)

{'It': 0, 'is': 1, 'worth': 2, 'noting': 3, 'that': 4, 'this': 5, 'road': 6, 'the': 7, 'main': 8, 'way': 9, 'of': 10, 'Rome': 11, '.': 12, 'Ginger': 13, 'hotel': 14, 'has': 15, 'opened': 16, 'here': 17, 'as': 18, 'well': 19, 'Use': 20, 'warm': 21, 'water': 22, 'for': 23, 'bathing': 24, 'In': 25, 'button': 26, 'hole': 27, 'surgery': 28, 'possibility': 29, 'hurting': 30, 'internal': 31, 'components': 32, 'abdomen': 33, 'much': 34, 'less': 35, 'Apart': 36, 'from': 37, 'some': 38, 'diseases': 39, 'parts': 40, 'chest': 41, 'may': 42, 'also': 43, 'be': 44, 'cured': 45, 'by': 46, 'method': 47, 'So': 48, ',': 49, 'why': 50, 'should': 51, 'it': 52, 'left': 53, 'eyes': 54, 'All': 55, 'social': 56, 'religious': 57, 'economic': 58, 'political': 59, 'etc': 60, 'activities': 61, 'city': 62, 'used': 63, 'to': 64, 'happen': 65, 'only': 66, 'Camping': 67, 'done': 68, 'through': 69, 'government': 70, 'non-government': 71, 'and': 72, 'voluntary': 73, 'organizations': 74, 'after': 75, 'comprehensive': 76,

In [0]:
def get_index_to_word_dict(vocab):
  index_to_word = {}
  for i, word in enumerate(vocab):
    index_to_word[i] = word
  return index_to_word  

In [0]:
src_index_to_word = get_index_to_word_dict(src_vocab)
dest_index_to_word = get_index_to_word_dict(dest_vocab)

In [17]:
print(src_index_to_word)
print(dest_index_to_word)

{0: 'It', 1: 'is', 2: 'worth', 3: 'noting', 4: 'that', 5: 'this', 6: 'road', 7: 'the', 8: 'main', 9: 'way', 10: 'of', 11: 'Rome', 12: '.', 13: 'Ginger', 14: 'hotel', 15: 'has', 16: 'opened', 17: 'here', 18: 'as', 19: 'well', 20: 'Use', 21: 'warm', 22: 'water', 23: 'for', 24: 'bathing', 25: 'In', 26: 'button', 27: 'hole', 28: 'surgery', 29: 'possibility', 30: 'hurting', 31: 'internal', 32: 'components', 33: 'abdomen', 34: 'much', 35: 'less', 36: 'Apart', 37: 'from', 38: 'some', 39: 'diseases', 40: 'parts', 41: 'chest', 42: 'may', 43: 'also', 44: 'be', 45: 'cured', 46: 'by', 47: 'method', 48: 'So', 49: ',', 50: 'why', 51: 'should', 52: 'it', 53: 'left', 54: 'eyes', 55: 'All', 56: 'social', 57: 'religious', 58: 'economic', 59: 'political', 60: 'etc', 61: 'activities', 62: 'city', 63: 'used', 64: 'to', 65: 'happen', 66: 'only', 67: 'Camping', 68: 'done', 69: 'through', 70: 'government', 71: 'non-government', 72: 'and', 73: 'voluntary', 74: 'organizations', 75: 'after', 76: 'comprehensive',

In [0]:
def reverse_train_sentences(data):
  l = data.values.tolist()
  t = []
  for item in l:
    p = item.split()
    p.reverse()
    p = " ".join(p)
    t.append(p)
  df = pd.DataFrame({'data':t})
  return df.data  

In [46]:
# Train - Test Split
X, y = lines.src, lines.dest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train_normal = copy.deepcopy(X_train)
X_train = reverse_train_sentences(X_train)
print(X_train.shape, X_test.shape)
X_train.sample(10)

(360,) (41,)


69     . incomplete is journey your , rulers Peshwa o...
21     . says Greerson Ian . Dr University Liverpool ...
137    . terminal the at bus another for wait to have...
160                         . month a for cough Frequent
130    . also years 15 to 10 for life normal a live p...
175                . you surprise will statue grand This
111    . applied being stitches no to due days more f...
199    . AIDS of inkling the give which up showing st...
195    . taken be can breakfast filling a of joy the ...
193    . seeing worth are it on made signs victory an...
Name: data, dtype: object

In [0]:
def generate_batch(X = X_train, y = y_train):
  batch_size = len(X)
  encoder_input_data = np.zeros((batch_size, src_max_len),dtype='float32')
  decoder_target_data = np.zeros((batch_size, dest_max_len, dest_vocab_len),dtype='float32')

  for i, (input_text, target_text) in enumerate(zip(X,y)):
      for t, word in enumerate(input_text.split()):
          encoder_input_data[i, t] = src_word_to_index[word]
      for t, word in enumerate(target_text.split()):
          decoder_target_data[i, t, dest_word_to_index[word]] = 1.
  return encoder_input_data, decoder_target_data

In [43]:
model = Sequential()

# Creating encoder network
model.add(Embedding(input_dim = src_vocab_len, output_dim = 300, input_length = src_max_len, mask_zero = True))
model.add(LSTM(units = 80))
model.add(RepeatVector(dest_max_len))

# Creating decoder network
# for _ in range(num_layers):
model.add(LSTM(units = 80, return_sequences=True))
model.add(TimeDistributed(Dense(dest_vocab_len)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 300)           560100    
_________________________________________________________________
lstm_3 (LSTM)                (None, 80)                121920    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 48, 80)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 48, 80)            51520     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 48, 2009)          162729    
_________________________________________________________________
activation_2 (Activation)    (None, 48, 2009)          0         
Total params: 896,269
Trainable params: 896,269
Non-trainable params: 0
________________________________________________

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 5

In [0]:
X,y = generate_batch(X_train, y_train)
xtest, ytest = generate_batch(X_test, y_test)

In [48]:
model.fit(X,y,
          steps_per_epoch = 1,
          epochs=epochs,
          validation_data =(xtest,ytest),
          validation_steps = 2)

Train on 360 samples, validate on 41 samples
Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 2.7323 - acc: 0.0022 - val_loss: 2.8116 - val_acc: 0.0203
Epoch 2/5
1/1 [==============================] - 0s 408ms/step - loss: 2.7303 - acc: 0.0212 - val_loss: 2.8085 - val_acc: 0.0203
Epoch 3/5
1/1 [==============================] - 0s 371ms/step - loss: 2.7269 - acc: 0.0207 - val_loss: 2.7978 - val_acc: 0.0203
Epoch 4/5
1/1 [==============================] - 0s 423ms/step - loss: 2.7149 - acc: 0.0203 - val_loss: 2.7530 - val_acc: 0.0203
Epoch 5/5
1/1 [==============================] - 0s 393ms/step - loss: 2.6660 - acc: 0.0203 - val_loss: 2.6683 - val_acc: 0.0203


In [0]:
yhat = model.predict(xtest, verbose=0)

In [51]:
print(np.shape(yhat))

(41, 48, 2009)
